<a href="https://colab.research.google.com/github/yousant/NewsClassification/blob/main/Multiclass_Classification_using_Transformer_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the Dataset

Pre-processing the raw data

Getting BERT Pre-trained model and its tokenizer

Training and evaluation

Prediction Pipeline

# **Loading the Dataset**

In [3]:
import pandas as pd
df = pd.read_csv("bbc-text.csv")
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [4]:
df.isnull().sum()

,0
category,0
text,0


# **Balacing Classes**

In [ ]:
# from imblearn.over_sampling import RandomOverSampler
# # Step 2: Initialize RandomOverSampler
# oversampler = RandomOverSampler(random_state=42)

# # Step 3: Fit RandomOverSampler to the data
# X = df['text'].values.reshape(-1, 1)  # Reshape to 2D array
# y = df['category']
# X_resampled, y_resampled = oversampler.fit_resample(X, y)

# # Step 4: Resample the data to balance the classes

# # Step 5: Split the resampled data into features and target columns
# df = pd.DataFrame({'text': X_resampled.flatten(), 'category': y_resampled})

In [5]:
df.shape

(2225, 2)

# **Train And Test Set**


In [6]:
from sklearn.model_selection import train_test_split


# Split the DataFrame into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

# Display the first few rows of each DataFrame
print("Train Set:")
print(df_train.shape)

print("\nTest Set:")
print(df_test.shape)


Train Set:
(1780, 2)

Test Set:
(445, 2)


In [7]:
df_train['category'].value_counts()

,count
category,
sport,413
business,409
politics,334
tech,319
entertainment,305


# **Cleaning** **Text**

In [ ]:
# import re
# import string
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer
# from nltk.tokenize import word_tokenize

# def preprocess_text(text):
#     # Lowercase the text
#     text = text.lower()

#     # Remove numbers
#     text = re.sub(r'\d+', '', text)

#     # Remove punctuation
#     text = text.translate(str.maketrans('', '', string.punctuation))

#     # Remove URLs
#     text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

#     # Remove emails
#     text = re.sub(r'\S*@\S*\s?', '', text)

#     # Remove emojis
#     emoji_pattern = re.compile("["
#                                u"\U0001F600-\U0001F64F"  # emoticons
#                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                                u"\U00002702-\U000027B0"
#                                u"\U000024C2-\U0001F251"
#                                "]+", flags=re.UNICODE)
#     text = emoji_pattern.sub(r'', text)

#     # Tokenize the text
#     tokens = word_tokenize(text)

#     # Remove stop words
#     stop_words = set(stopwords.words('english'))
#     tokens = [word for word in tokens if word not in stop_words]

#     # Stemming
#     stemmer = PorterStemmer()
#     tokens = [stemmer.stem(word) for word in tokens]

#     # Join tokens back into a single string
#     text = ' '.join(tokens)

#     return text


# df['clean_text'] = df['text'].apply(lambda x: preprocess_text(x))

# **Converting our Target column into Categorical data**

In [8]:
df_test['category'].value_counts()

,count
category,
business,101
sport,98
politics,83
tech,82
entertainment,81


In [9]:
df_test.shape

(445, 2)

In [10]:
encoded_dict = {"sport":0,"business":1, 'politics':2, "entertainment":3,'tech':4}
df_train['category'] = df_train['category'].map(encoded_dict)

In [11]:
df_test['category'] = df_test['category'].map(encoded_dict)

In [12]:
from keras.utils import to_categorical
# Convert the 'category' column to categorical values
y_train = to_categorical(df_train['category'])

In [13]:
y_test = to_categorical(df_test['category'])

In [14]:
y_train.shape, y_test.shape

((1780, 5), (445, 5))

# **Loading Model and Tokenizer from the transformers package**

In [15]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [16]:
max_len_train = max(len(text) for text in df_train['text'])
max_len_test= max(len(text) for text in df_test['text'])

In [17]:
max_len_train, max_len_test

(19136, 25483)

# **Input Data Modeling**

Before training, we need to convert the input textual data into BERT’s input data format using a tokenizer.

Since we have loaded bert-base-cased, so tokenizer will also be Bert-base-cased.

In [18]:
 # Tokenize the input
max_len = 70
x_train = tokenizer(
    text=df_train.text.tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)
x_test = tokenizer(
    text=df_test.text.tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

Tokenizer takes all the necessary parameters and returns tensor in the same format Bert accepts.

return_token_type_ids = False: token_type_ids is not necessary for our training in this case.

return_attention_mask = True we want to include attention_mask in our input.

return_tensors=’tf’: we want our input tensor for the TensorFlow model.

max_length=70: we want the maximum length of each sentence to be 70; if a sentence is bigger than this, it will be trimmed if a sentence is smaller than 70 then it will be padded.

add_special_tokens=True, CLS, SEP token will be added in the tokenization.

Hereafter data modelling, the tokenizer will return a dictionary (x_train) containing ‘Input_ids’, ‘attention_mask’ as key for their respective

In [19]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [20]:
print("input ids:",input_ids.shape)
print("attention_mask:", attention_mask.shape)

input ids: (1780, 70)
attention_mask: (1780, 70)


# **Model Building**

**Importing necessary libraries.**

In [21]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [22]:
# Define input layers
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

input_ids.shape,input_mask.shape

((None, 70), (None, 70))

In [26]:
import tensorflow as tf
from tensorflow.keras import layers

# Define a custom Keras layer that wraps the BERT model call
class BertLayer(layers.Layer):
    def __init__(self, bert_model):
        super(BertLayer, self).__init__()
        self.bert = bert_model

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]
        return outputs

# Assuming 'bert' is your preloaded BERT model
input_ids = layers.Input(shape=(70,), dtype=tf.int32, name='input_ids')
input_mask = layers.Input(shape=(70,), dtype=tf.int32, name='attention_mask')

# Use the custom BERT layer in your Keras model
embeddings = BertLayer(bert)([input_ids, input_mask])
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = tf.keras.layers.Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)

# Define the final model
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=out)

# Compile and use the model as needed
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


In [29]:
# Define model architecture (assuming 'bert' is already defined)
embeddings = bert(input_ids, attention_mask=input_mask)[0]
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation='relu')(out)
y = Dense(5, activation='sigmoid')(out)

# Create the model
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

ValueError: Exception encountered when calling layer 'tf_bert_model' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tf_bert_model' (type TFBertModel):
  • input_ids=<KerasTensor shape=(None, 70), dtype=int32, sparse=None, name=input_ids>
  • attention_mask=<KerasTensor shape=(None, 70), dtype=int32, sparse=None, name=attention_mask>
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

Bert layers accept three input arrays, input_ids, attention_mask, token_type_ids

input_ids means our input words encoding, then attention mask,

token_type_ids is necessary for the question-answering model; in this case, we will not pass token_type_ids.

For the Bert layer, we need two input layers, in this case, input_ids, attention_mask.

Embeddings contain hidden states of the Bert layer. using

GlobalMaxPooling1D then dense layer to build CNN layers using hidden states of Bert. These CNN layers will yield our output. bert[0] is the last hidden state, bert[1] is the pooler_output, for building CNN layers on top of the BERT layer, we have used Bert’s hidden forms.


# **Model Compilation**

Defining learning parameters and compiling the model.

learning_rate = 5e-05 the learning rate for the model will be significantly lower.

Loss = CategoricalCrossentropy since we are passing the categorical data as the target.

Balanced accuracy will take care of our average accuracy for all the classes.

In [30]:
from keras.optimizers import Adam
from keras.optimizers.schedules import ExponentialDecay
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy

# Define learning rate schedule
initial_learning_rate = 5e-05
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,  # Adjust this value according to your needs
    decay_rate=0.01,    # Adjust this value according to your needs
    staircase=True)
# Define optimizer, loss, and metrics
optimizer = Adam(
    learning_rate=lr_schedule,
    epsilon=1e-08,
    clipnorm=1.0
)
loss = CategoricalCrossentropy(from_logits=True)
metric = CategoricalAccuracy(name='balanced_accuracy')

# Compile the model
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metric
)

ValueError: Expected `metrics` argument to be a list, tuple, or dict. Received instead: metrics=<CategoricalAccuracy name=balanced_accuracy> of type <class 'keras.src.metrics.accuracy_metrics.CategoricalAccuracy'>

# **Model Training**

You have the model ready with x_train, y_train. You can now train the model.

Training and fine-tuning of the BERT model takes a bit longer time. so be Patience.

model.fit returns a history object which keeps all the training history. x_test became a dictionary containing ‘input_ids’, ‘attention_mask‘ after pre-processing. We are passing input_ids and attention_mask for the training. In the validation data, we are passing the test data.

In [31]:
train_history = model.fit(
    x={'input_ids': x_train['input_ids'], 'attention_mask': x_train['attention_mask']},
    y=y_train,
    validation_data=(
        {'input_ids': x_test['input_ids'], 'attention_mask': x_test['attention_mask']},
        y_test
    ),
    epochs=2,
    batch_size=36
)


Epoch 1/2


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 5), output.shape=(None, 128)

# **Model Evaluation**

Testing our model on the test data.

In [32]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

14/14 ━━━━━━━━━━━━━━━━━━━━ 113s 8s/step


array([0.        , 0.        , 0.38961884, 0.6854384 , 0.15671086,
       0.        , 0.8956196 , 0.        , 0.28208554, 0.        ,
       0.9770241 , 0.        , 0.6169921 , 1.670728  , 1.4725311 ,
       0.        , 0.        , 1.5533596 , 0.        , 0.        ,
       0.07059914, 1.3129145 , 0.57958347, 1.4837799 , 0.        ,
       0.        , 0.        , 0.        , 0.2948984 , 0.        ,
       0.6979672 , 0.        , 0.950359  , 0.1274672 , 0.8674567 ,
       0.07854784, 1.0850606 , 1.6414415 , 0.        , 1.1286938 ,
       0.        , 0.23197427, 0.        , 0.41508216, 0.2870273 ,
       0.        , 0.01198655, 0.33493254, 0.        , 1.2089355 ,
       0.20615855, 0.49714288, 0.        , 0.3842356 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 1.1660849 , 0.91622996,
       0.77014446, 0.        , 0.7577292 , 0.        , 0.        ,
       0.8518168 , 0.        , 1.6344671 , 0.        , 1.55533

In [33]:
# Taking the index of value having maximum probability.
import numpy as np
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = df_test.category
y_true

,category
414,2
420,1
1644,3
416,4
1232,0
...,...
741,1
205,1
1102,1
668,1


In [34]:
# Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      98.0
           1       0.00      0.00      0.00     101.0
           2       0.00      0.00      0.00      83.0
           3       0.00      0.00      0.00      81.0
           4       0.00      0.00      0.00      82.0
          17       0.00      0.00      0.00       0.0
          74       0.00      0.00      0.00       0.0
          77       0.00      0.00      0.00       0.0
          90       0.00      0.00      0.00       0.0
         102       0.00      0.00      0.00       0.0
         117       0.00      0.00      0.00       0.0

    accuracy                           0.00     445.0
   macro avg       0.00      0.00      0.00     445.0
weighted avg       0.00      0.00      0.00     445.0



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

# **Prediction Pipeline**

Converting indexes back to the Sentiment label:


In [36]:
def predict(text):
    x_val = tokenizer(
        text=texts,
        add_special_tokens=True,
        max_length=70,
        truncation=True,
        padding='max_length',
        return_tensors='tf',
        return_token_type_ids=False,
        return_attention_mask=True,
        verbose=True
    )
    validation = model.predict({'input_ids': x_val['input_ids'], 'attention_mask': x_val['attention_mask']}) * 100

    # Create lists to store the results
    labels = []
    scores = []

    # Iterate over the predicted values and store them
    for key, value in zip(encoded_dict.keys(), validation[0]):
        labels.append(key)
        scores.append(value)

    # Get the predicted label with the highest score
    predicted_label = labels[scores.index(max(scores))]

    # Return the labels, scores, and the predicted label with its score
    return labels, scores, predicted_label, max(scores)


In [37]:
texts = 'input the textbrown and blair face new rift claims for the umpteenth time  tony blair and gordon brown are said to have declared all out war on each other.  this time the alleged rift is over who should take the credit for the government s global aid and debt initiatives  particularly in the wake of the tsunami disaster - an issue many hoped and believed was above such things. it dominated the prime minister s monthly news conference  which saw mr blair start in full irritation mode as he was forced to bat away question after question about his relationship with his neighbour. as he told journalists:  i am not interested in what goes in and out of newspapers. there is a complete unity of purpose.  and he again heaped praise on mr brown saying he was doing a great job  and would continue doing it - although he would not commit to any job for mr brown after the election.  so why did he arrange his press conference at the last moment so it coincided with mr brown s long-arranged keynote speech on aid and debt  he was asked  by now mr blair had moved from irritation mode to his barely disguised fury setting. he snapped back that the hacks knew very well what the operational reasons were for the timing of his press conference. well  not really  as it happens.  and he repeated what a great man gordon was and how united they were  before again sneering that he took absolutely no notice of what went in and out of the newspapers  preferring to get on with the job of doing the best for the country and the world. although in the next breath he declared:  i get increasingly alarmed by what i read in the newspapers  before catching himself on and quickly adding:  in so far as i read them of course.  he probably had good reason to be alarmed because the newspapers had been full of stories about the claimed open warfare between the two men.  as far as the timing of the prime minister s press conference is concerned  there are two options. the first is that it was a calculated attempt to upstage the chancellor and seize back the initiative on the big issue of the moment. if that is the case it suggests that even the fear of seriously negative newspaper headlines is not enough to stop the squabbling. the second option is that it was an unavoidable coincidence  which would suggest the government has lost its once-famed ability to strictly co-ordinate announcements - through the infamous downing street grid - to avert just such allegations.  either way  the effect was the same - to overshadow the big announcements of government policy on a hugely pertinent issue. and there had been previous suggestions that the new year had started with a fresh outbreak of the warfare between the two men. firstly  the prime minister insisted on wednesday that he had been intimately involved in the development of the proposals to get g8 countries to freeze debt repayments from the tsunami-hit countries. it was claimed he had been embarrassed by the fact that gordon brown appeared to have taken the initiative over the government s response to the disaster while mr blair was still on holiday in egypt.  then  as if to pour fuel on the flames  both men separately spoke about working on tsunami or wider aid and development policy with their cabinet colleagues foreign secretary jack straw  aid minister hilary benn and deputy prime minister john prescott - without mentioning the other. all this came amid fresh claims that mr brown was still seething that he had been excluded from a prominent role in general election planning and had  as a result  started to set out his own platform. the fact that he used an article in the guardian newspaper to set out what he believed  should  be in the manifesto  has embarked on a mini tour of britain to set out his aid plans and will next week visit africa on the same mission - often seen as the prime minister s  turf  - has only added to the impression of rival camps operating entirely independently of each other. the prime minister denied all that as well  repeating his insistence that it was inconceivable the economy and the chancellor would not be at the centre of the election campaign. but the big fear with many on the labour benches now is that  unless a lid can be put on the speculation over the rivalry  it may even threaten to undermine the election campaign itself.'

labels, scores, predicted_label, pred_prob = predict(texts)

for name, score in zip(labels,scores):
  print("Labels : ", name, " Scores : ", score)

print("====================================================")
print("predicted_label : ", predicted_label, " Scores : ", pred_prob)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step
Labels :  sport  Scores :  20.804136
Labels :  business  Scores :  0.0
Labels :  politics  Scores :  31.727928
Labels :  entertainment  Scores :  50.923794
Labels :  tech  Scores :  20.222769
predicted_label :  entertainment  Scores :  50.923794


In [38]:
texts="housewives lift channel 4 ratings the debut of us television hit desperate housewives has helped lift channel 4 s january audience share by 12% compared to last year.  other successes such as celebrity big brother and the simpsons have enabled the broadcaster to surpass bbc two for the first month since last july. bbc two s share of the audience fell from 11.2% to 9.6% last month in comparison with january 2004. celebrity big brother attracted fewer viewers than its 2002 series.  comedy drama desperate housewives managed to pull in five million viewers at one point during its run to date  attracting a quarter of the television audience. the two main television channels  bbc1 and itv1  have both seen their monthly audience share decline in a year on year comparison for january  while five s proportion remained the same at a slender 6.3%. digital multi-channel tv is continuing to be the strongest area of growth  with the bbc reporting freeview box ownership of five million  including one million sales in the last portion of 2004. its share of the audience soared by 20% in january 2005 compared with last year  and currently stands at an average of 28.6%."
labels, scores, predicted_label, pred_prob = predict(texts)

for name, score in zip(labels,scores):
  print("Labels : ", name, " Scores : ", score)

print("====================================================")
print("predicted_label : ", predicted_label, " Scores : ", pred_prob)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step
Labels :  sport  Scores :  0.0
Labels :  business  Scores :  4.6287413
Labels :  politics  Scores :  9.933932
Labels :  entertainment  Scores :  55.26192
Labels :  tech  Scores :  43.851376
predicted_label :  entertainment  Scores :  55.26192
